In [1]:
import csv
import datetime
import requests
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import yfinance as yf

## Data Extraction

In [2]:
source_file = 'Data Sources.csv'

class Variables:
    def __init__(self, df):
        self.dep = df[df['Dependent']=='Y']['Name'].tolist()

        self.indep = df[df['Dependent']!='Y']['Name'].tolist()
        self.indep = [i for i in self.indep if 'business expectations' not in i.lower()]

        self.freq = dict(zip(df['Name'],df['Frequency']))

        self.non_stat = []

def get_dos(url):
    response = requests.get(url=url).json()
    return({i['Key']:float(i['Value']) for i in response})

def get_sgx(url):
    data = {}
    response = requests.get(url=url).json()
    for d in response['data']:
        date = d['trading_time'].split('_')[0]
        date = datetime.datetime.strptime(date, '%Y%m%d').date()
        data[date] = float(d['lp'])
    return(data)

def get_mas(url, params={'fields':'end_of_month,m1','limit':100,'offset':0}):
    response = requests.get(url=url, params=params).json()

    no_records = int(response['result']['total'])
    pagesize = int(response['result']['limit'])
    pages = no_records//pagesize + (no_records%pagesize>0)

    # parse first page
    data = {}
    for i in response['result']['records']:
        data[i['end_of_month']] = float(i['m1'])

    # parse rest of data to get complete series
    for i in range(1,pages):
        params['offset'] = i*100
        response = requests.get(url=url, params=params).json()
        for i in response['result']['records']:
            data[i['end_of_month']] = float(i['m1'])

    return(data)

sources = pd.read_csv(source_file, encoding='utf-8')
variables = Variables(sources)

requests_funcs = {'mas':get_mas, 'sgx':get_sgx, 'singstat':get_dos}

apis = {'mas':[], 'sgx':[], 'singstat':[]}
for k in apis.keys():
    for i in zip(sources['Name'], sources['Frequency'], sources['API']):
        if k in i[-1]:
            apis[k].append({'Name':i[0], 'Frequency':i[1], 'API':i[2]})
            
# pull data
ts_data = {}
for k,v in apis.items():
    for dim in v:
        ts_data[dim['Name']] = requests_funcs[k](dim['API'])

In [4]:
list(ts_data)

['M1 Money Supply',
 'Straits Times Index (STI)',
 'Gross Domestic Product, Year On Year Growth Rate | GDP In Chained (2015) Dollars',
 'Composite Leading Index (2015 = 100) | Quarterly Composite Leading Index',
 'Business Expectations Of The Manufacturing Sector | General Business Expectations (Forecast For Next 6 Months) ',
 'Business Expectations For The Services Sector - Operating Receipts Forecast For The Next Quarter, Weighted Percentages Of Up, Same, Down | Net Weighted Balance - Total Services Sector',
 'Business Expectations For The Services Sector - General Business Outlook For The Next 6 Months, Weighted Percentages Of Up, Same, Down | Net Weighted Balance - Total Services Sector',
 'Domestic Wholesale Trade Index, (2017 = 100), In Chained Volume Terms | Total',
 'Foreign Wholesale Trade Index, (2017 = 100), In Chained Volume Terms | Total',
 'Business Receipts Index For Services Industries, (2014=100) | Total Services',
 'Food & Beverage Services Index, (2017 = 100), In Cha

### Combining Data Sources

#### M1 Money Supply

In [5]:
# observe m1 money supply
keys = list(ts_data['M1 Money Supply'].keys())
values = list(ts_data['M1 Money Supply'].values())

# Print the last 5 key-value pairs
for i in range(len(keys)-5, len(keys)):
    print(keys[i], values[i])

2021-02 262747.3
2021-03 743486.3
2021-04 271602.0
2021-05 272111.3
2021-06 279298.0


The MAS M1 Money Supply api abruptly stops at 2021-06, we can append the latest M1 Money Supply by downloading from MAS directly

In [6]:
add_money_supply = {}

with open('Money and Banking - I.1.csv', 'r') as file:
    csv_reader = csv.reader(file)
    # Skip header row and row 1 based on the CSV structure
    next(csv_reader)
    next(csv_reader)  
    
    for row in csv_reader:
        if len(row) >= 2:
            date_str = row[0]
            date_obj = datetime.datetime.strptime(date_str, '%Y %b')
            formatted_date = date_obj.strftime('%Y-%m')
            value = float(row[1])
            add_money_supply[formatted_date] = value

print(add_money_supply)

{'2021-07': 276525.0, '2021-08': 277449.2, '2021-09': 279007.1, '2021-10': 280696.5, '2021-11': 285287.7, '2021-12': 286176.3, '2022-01': 287058.9, '2022-02': 288626.0, '2022-03': 292648.7, '2022-04': 291430.7, '2022-05': 288637.0, '2022-06': 283812.9, '2022-07': 278259.4, '2022-08': 274380.7, '2022-09': 268827.6, '2022-10': 261561.5, '2022-11': 260165.8, '2022-12': 255547.3, '2023-01': 250174.8, '2023-02': 247795.7, '2023-03': 252109.9, '2023-04': 252203.4}


In [7]:
ts_data['M1 Money Supply'].update(add_money_supply)

In [8]:
print(ts_data['M1 Money Supply'])

{'1991-01': 14844.5, '1991-02': 14978.1, '1991-03': 14936.7, '1991-04': 14662.3, '1991-05': 14716.2, '1991-06': 14633.2, '1991-07': 16015.3, '1991-08': 14561.1, '1991-09': 14720.5, '1991-10': 14966.5, '1991-11': 15275.9, '1991-12': 16430.0, '1992-01': 16795.8, '1992-02': 15972.5, '1992-03': 16694.4, '1992-04': 16322.1, '1992-05': 16742.7, '1992-06': 17033.2, '1992-07': 17238.8, '1992-08': 17125.6, '1992-09': 17905.5, '1992-10': 17905.0, '1992-11': 17850.4, '1992-12': 18515.6, '1993-01': 18733.0, '1993-02': 18433.1, '1993-03': 18769.6, '1993-04': 19240.5, '1993-05': 19226.6, '1993-06': 19219.3, '1993-07': 19273.4, '1993-08': 19564.0, '1993-09': 19978.3, '1993-10': 25499.3, '1993-11': 22057.0, '1993-12': 22882.2, '1994-01': 22712.5, '1994-02': 22069.8, '1994-03': 22491.1, '1994-04': 21531.4, '1994-05': 21982.3, '1994-06': 22396.9, '1994-07': 22073.7, '1994-08': 22432.0, '1994-09': 23139.3, '1994-10': 23451.0, '1994-11': 23114.4, '1994-12': 23411.5, '1995-01': 25164.6, '1995-02': 23334.1,

#### STI Index

In [9]:
# observe STI
keys = list(ts_data['Straits Times Index (STI)'].keys())
values = list(ts_data['Straits Times Index (STI)'].values())

# Print the first 5 key-value pairs
for i in range(0, 6):
    print(keys[i], values[i])

2011-01-04 3250.29
2011-01-05 3254.25
2011-01-06 3279.7
2011-01-07 3261.36
2011-01-10 3229.27
2011-01-11 3241.5


In [11]:
# instead a data source from yfinance instead
# Straits Times Index symbol
ticker_symbol = "^STI"  # Straits Times Index symbol

start_date = "1900-01-01"
end_date = "2023-12-31"

# Fetch the historical data
sti = yf.download(ticker_symbol, start=start_date, end=end_date)

adj_close_dict = sti['Adj Close'].round(1).to_dict()
ts_data['Straits Times Index (STI)'] = {str(key.date()): value for key, value in adj_close_dict.items()}

[*********************100%***********************]  1 of 1 completed


## Save Raw Data CSV

In [12]:
# pass data into pandas series
ts_pd = {}
for k,v in apis.items():
    for dim in v:

        if dim['Frequency']=='Q' and k=='singstat':
            periods = [p.split()[0]+p.split()[-1][::-1] for p in list(ts_data[dim['Name']])]
            periods = pd.PeriodIndex(periods, freq=dim['Frequency'])
            ts_pd[dim['Name']] = pd.Series(ts_data[dim['Name']].values(), index=periods)

        elif dim['Frequency']=='M':
            periods = pd.to_datetime(list(ts_data[dim['Name']])) + pd.tseries.offsets.MonthEnd(0)
            ts_pd[dim['Name']] = pd.Series(ts_data[dim['Name']].values(), index=periods)

        elif dim['Frequency']=='D' and k=='sgx':
            periods = pd.to_datetime(list(ts_data[dim['Name']]))
            ts_pd[dim['Name']] = pd.Series(ts_data[dim['Name']].values(), index=periods).resample('M').last()
            variables.freq[dim['Name']] = 'M' # update to 'M' since resampled

In [13]:
ts_pd_copy = copy.deepcopy(ts_pd)

In [14]:
# resample all series to monthly and start from first valid index
for series, freq in zip(sources['Name'], sources['Frequency']):

    if freq=='Q':
        ts_pd_copy[series] = ts_pd_copy[series].resample('M', convention='end').asfreq()
        ts_pd_copy[series].index = pd.to_datetime(ts_pd_copy[series].index.strftime('%Y-%m-%d'))

    if ts_pd_copy[series].index[0]!=ts_pd_copy[series].first_valid_index():
        ts_pd_copy[series] = ts_pd_copy[series][ts_pd_copy[series].first_valid_index():]

In [15]:
# pass all series into dataframe, start dataframe from first year of GDP growth data
ts_df = pd.DataFrame(ts_pd_copy)
ts_df = ts_df.loc[ts_df.index.year>=ts_df.loc[:,variables.dep[0]].first_valid_index().year]

# rearrange columns in correct order for factor modelling as explained in markdown above
ts_df = ts_df[variables.dep+\
              [k for k,v in variables.freq.items() if v=='M' and k!=variables.dep[0]]+\
              [k for k,v in variables.freq.items() if v=='Q' and k!=variables.dep[0]]]
ts_df.head()

,"Gross Domestic Product, Year On Year Growth Rate | GDP In Chained (2015) Dollars","Food & Beverage Services Index, (2017 = 100), In Chained Volume Terms | Total","Retail Sales Index, (2017 = 100), In Chained Volume Terms | Total",Sea Cargo And Shipping Statistics (Total Cargo) | Total Container Throughput (Thousand Twenty-Foot Equivalent Units),Air Cargo Tonnage | Total Direct Tonnage (Tonne),Domestic Exports By Commodity Division | Total Domestic Exports,Merchandise Imports By Commodity Division | Total Merchandise Imports,Straits Times Index (STI),M1 Money Supply,Index Of Industrial Production (2019 = 100) | Total,...,Business Expectations Of The Manufacturing Sector | General Business Expectations (Forecast For Next 6 Months),"Business Expectations For The Services Sector - Operating Receipts Forecast For The Next Quarter, Weighted Percentages Of Up, Same, Down | Net Weighted Balance - Total Services Sector","Business Expectations For The Services Sector - General Business Outlook For The Next 6 Months, Weighted Percentages Of Up, Same, Down | Net Weighted Balance - Total Services Sector","Domestic Wholesale Trade Index, (2017 = 100), In Chained Volume Terms | Total","Foreign Wholesale Trade Index, (2017 = 100), In Chained Volume Terms | Total","Business Receipts Index For Services Industries, (2014=100) | Total Services",Tourism Receipts By Major Components (Year-To-Date) | Tourism Receipts,Average Monthly Nominal Earnings Per Employee | Overall Economy,Job Vacancies By Industry And Occupational Group (SSIC 2020) (End Of Period) | Total,Unemployment Rate (End Of Period) | Total Unemployment Rate
1976-01-31,NaN,NaN,NaN,NaN,NaN,783226.0,1830979.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-02-29,NaN,NaN,NaN,NaN,NaN,700088.0,1658989.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-03-31,8.2,NaN,NaN,NaN,NaN,715228.0,1781336.0,NaN,NaN,NaN,...,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-04-30,NaN,NaN,NaN,NaN,NaN,777443.0,1942837.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-05-31,NaN,NaN,NaN,NaN,NaN,720721.0,1462480.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
ts_df.to_csv('raw_data.csv')

## Stationary Test and Transformation

In [17]:
# if p-value >0.05, var is non-stationary
for i in variables.indep:

    # if unit root, apply transformation, take % yoy growth (which also removes seasonality)
    if adfuller(ts_pd[i])[1]>0.05:
        print(f'[Non-stationary] {i}')
        if variables.freq[i]=='M':
            ts_pd[i] = ts_pd[i].pct_change(periods=12) * 100
        elif variables.freq[i]=='Q':
            ts_pd[i] = ts_pd[i].pct_change(periods=4) * 100
        variables.non_stat.append(i)

    else:
        print(f'[Stationary] {i}')

[Non-stationary] Composite Leading Index (2015 = 100) | Quarterly Composite Leading Index
[Non-stationary] Domestic Wholesale Trade Index, (2017 = 100), In Chained Volume Terms | Total
[Non-stationary] Foreign Wholesale Trade Index, (2017 = 100), In Chained Volume Terms | Total
[Non-stationary] Business Receipts Index For Services Industries, (2014=100) | Total Services
[Stationary] Food & Beverage Services Index, (2017 = 100), In Chained Volume Terms | Total
[Non-stationary] Retail Sales Index, (2017 = 100), In Chained Volume Terms | Total
[Stationary] Tourism Receipts By Major Components (Year-To-Date) | Tourism Receipts
[Non-stationary] Sea Cargo And Shipping Statistics (Total Cargo) | Total Container Throughput (Thousand Twenty-Foot Equivalent Units)
[Stationary] Air Cargo Tonnage | Total Direct Tonnage (Tonne)
[Non-stationary] Domestic Exports By Commodity Division | Total Domestic Exports
[Non-stationary] Merchandise Imports By Commodity Division | Total Merchandise Imports
[Non-

In [18]:
# resample all series to monthly and start from first valid index
for series, freq in zip(sources['Name'], sources['Frequency']):

    if freq=='Q':
        ts_pd[series] = ts_pd[series].resample('M', convention='end').asfreq()
        ts_pd[series].index = pd.to_datetime(ts_pd[series].index.strftime('%Y-%m-%d'))

    if ts_pd[series].index[0]!=ts_pd[series].first_valid_index():
        ts_pd[series] = ts_pd[series][ts_pd[series].first_valid_index():]

In [19]:
# pass all series into dataframe, start dataframe from first year of GDP growth data
ts_transformed_df = pd.DataFrame(ts_pd)
ts_transformed_df = ts_transformed_df.loc[ts_transformed_df.index.year>=ts_transformed_df.loc[:,variables.dep[0]].first_valid_index().year]

# rearrange columns in correct order for factor modelling as explained in markdown above
ts_transformed_df = ts_transformed_df[variables.dep+\
              [k for k,v in variables.freq.items() if v=='M' and k!=variables.dep[0]]+\
              [k for k,v in variables.freq.items() if v=='Q' and k!=variables.dep[0]]]

ts_transformed_df.head()

,"Gross Domestic Product, Year On Year Growth Rate | GDP In Chained (2015) Dollars","Food & Beverage Services Index, (2017 = 100), In Chained Volume Terms | Total","Retail Sales Index, (2017 = 100), In Chained Volume Terms | Total",Sea Cargo And Shipping Statistics (Total Cargo) | Total Container Throughput (Thousand Twenty-Foot Equivalent Units),Air Cargo Tonnage | Total Direct Tonnage (Tonne),Domestic Exports By Commodity Division | Total Domestic Exports,Merchandise Imports By Commodity Division | Total Merchandise Imports,Straits Times Index (STI),M1 Money Supply,Index Of Industrial Production (2019 = 100) | Total,...,Business Expectations Of The Manufacturing Sector | General Business Expectations (Forecast For Next 6 Months),"Business Expectations For The Services Sector - Operating Receipts Forecast For The Next Quarter, Weighted Percentages Of Up, Same, Down | Net Weighted Balance - Total Services Sector","Business Expectations For The Services Sector - General Business Outlook For The Next 6 Months, Weighted Percentages Of Up, Same, Down | Net Weighted Balance - Total Services Sector","Domestic Wholesale Trade Index, (2017 = 100), In Chained Volume Terms | Total","Foreign Wholesale Trade Index, (2017 = 100), In Chained Volume Terms | Total","Business Receipts Index For Services Industries, (2014=100) | Total Services",Tourism Receipts By Major Components (Year-To-Date) | Tourism Receipts,Average Monthly Nominal Earnings Per Employee | Overall Economy,Job Vacancies By Industry And Occupational Group (SSIC 2020) (End Of Period) | Total,Unemployment Rate (End Of Period) | Total Unemployment Rate
1976-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-03-31,8.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
ts_transformed_df.to_csv('transformed.csv')